In [84]:
import os, sys
from collections import namedtuple
from functools import partial
import itertools

current = os.path.dirname(os.path.realpath(''))
parent = os.path.dirname(current)
sys.path.append(parent)
print(current+'/algorithms')

sys.path.insert(0, current+'/algorithms')

from geneticalgo import *

/Users/davidfandrei/Desktop/UEvry/08_Algorithms/Project1/algorithms


#### This notebook shows the application of the algorithms to the Knapsack problem

In [49]:
Item = namedtuple('Item', ['name', 'value', 'weight'])

items = [
    Item('Laptop', 500, 2200),
    Item('Headphones', 150, 160),
    Item('Coffee Mug', 60, 350),
    Item('Notepad', 40, 333),
    Item('Water Bottle', 30, 192),
]

#### In the following, the functions for item generation and the fitness function are defined:

In [51]:
def generate_items(num: int):
    return [Item(f"item{i}", i, i) for i in range(1, num+1)]

def genome_to_items(genome, items):
    result = []
    for i, item in enumerate(items):
        if genome[i] == 1:
            result += [item.name]

    return result

def fitness(genome, items, weight_limit: int) -> int:
    if len(genome) != len(items):
        raise ValueError("genome and items must be of same length")

    weight = 0
    value = 0

    for i, item in enumerate(items):
        if genome[i] == 1:
            weight += item.weight
            value += item.value

            if weight > weight_limit:
                return 0

    return value

In [58]:
def test(result: list, items):
    total_weight = 0
    total_value = 0
    for i, item in enumerate(items):
        if result[i] == 1:
            total_weight += item.weight
            total_value += item.value
    return total_weight, total_value

#### For the small collection of items, a brute force solution can be tried:

In [93]:
def brute_force(items, fitness_func, weight_lim, fitness_lim):
    
    comb = [list(i) for i in itertools.product([0, 1], repeat=len(items))]
    best = [0 for item in items]
    max_value = 0

    for sol in comb:
        value = fitness_func(sol, items, weight_lim)
        if value <= fitness_lim and value > max_value:
            best = sol
            max_value = value

    return best

weight_lim = 3000
fitness_lim = 740

res = brute_force(items, fitness, weight_lim, fitness_lim)
weight, value = test(res, items)

print(f"Best solution: {genome_to_items(res, items)}")
print(f"weight: {weight}, value: {value}")


Best solution: ['Laptop', 'Headphones', 'Coffee Mug', 'Water Bottle']
weight: 2902, value: 740


#### Next, the Genetic Algorithm is used:

In [94]:
population, generations = run_evolution(
    populate_func = partial(generate_population, size=10, genome_length = len(items)),
    fitness_func = partial(fitness, items=items, weight_limit=3000),
    fitness_limit = 740,
    generation_limit = 100,
    elitism = True
    )

print(f"Number generations: {generations}")
print(f"Best solution: {genome_to_items(population[0], items)}")

w, v = test(population[0], items)
print(f"weight: {w}, value: {v}")

Number generations: 15
Best solution: ['Laptop', 'Headphones', 'Coffee Mug', 'Water Bottle']
weight: 2902, value: 740
